In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

data = pd.read_csv('../networks/data/global_before_1700.csv', index_col=[0])
data = data[~((data['wikidata_id']=='Q1237726')&(data['region_name']=='Arabic world'))]
data = data[~((data['wikidata_id']=='Q7413013')&(data['region_name']=='Arabic world'))]

df_duplicated = data[data['meta_region'].isin(['non_europe', 'europe'])]
duplicates = df_duplicated[df_duplicated.duplicated('wikidata_id', keep=False)]


df_test = data[(data['region_name']=='German world')&(data['meta_occupation'].str.contains('anatomist'))]
len(set(df_test['wikidata_id']))




54

In [2]:
df = data.copy()

# Remove those for original graph with original datasets
bigger_regions = {'Central Europe':'Eastern Europe', 
                'Slav world':'Eastern Europe',
                'Balkans':'Eastern Europe',
                'Korea':'Asia',
                'Chinese world':'Asia', 
                'Japan':'Asia', 
                'Indian world':'Asia',
                'South East Asia':'Asia', 
                'Greek World':'Ancient Mediterranean', 
                'Latin World':'Ancient Mediterranean',
                'Arabic world':'MENA',
                'Persian world':'MENA',
                'Ottoman Turkey':'MENA',
                'Spain':'Iberian Peninsula',
                'Portugal':'Iberian Peninsula'}

df['meta_region_name'] = df['region_name'].apply(lambda x: bigger_regions.get(x, x))
df['meta_occupation'] = df['meta_occupation'].apply(lambda x : x.split(' | '))
df = df.explode('meta_occupation')



In [3]:


df = df.drop('meta_region', axis=1)
df = df.drop_duplicates()
df_test = df[(df['region_name']=='German world')&(df['meta_occupation']=='anatomist')].sort_values('wikidata_id')

In [4]:



df = df.groupby(['meta_occupation','meta_region_name'])['wikidata_id'].count().reset_index()


In [5]:
df[(df['meta_occupation']=='anatomist')&(df['meta_region_name']=='Ancient Mediterranean')]

,meta_occupation,meta_region_name,wikidata_id
0,anatomist,Ancient Mediterranean,4


In [6]:
df_eu_non_eu = data.copy()
df_eu_non_eu['meta_region_name'] = df_eu_non_eu['region_name'].apply(lambda x: bigger_regions.get(x, x))
df_eu_non_eu['meta_occupation'] = df_eu_non_eu['meta_occupation'].apply(lambda x : x.split(' | '))
df_eu_non_eu = df_eu_non_eu.explode('meta_occupation')
df_eu_non_eu[(df_eu_non_eu['meta_occupation']=='anatomist')&(df_eu_non_eu['meta_region']=='non_europe')]

df_eu_non_eu = df_eu_non_eu.groupby(['meta_occupation','meta_region'])['wikidata_id'].count().reset_index()
df_eu_non_eu = df_eu_non_eu.rename(columns={'meta_region':'meta_region_name'})

final_df = pd.concat([df, df_eu_non_eu])
final_df = final_df.reset_index(drop=True)

In [7]:
pivot_table = final_df.pivot_table(index='meta_occupation', columns='meta_region_name', values='wikidata_id', fill_value=0)
pivot_table = pivot_table.drop('antiquity', axis=1)
pivot_table['Total number of scholars'] = pivot_table['europe'] + pivot_table['non_europe']
#pivot_table = pivot_table.sort_values('total', ascending=False)




In [8]:


data_unique = data[['wikidata_id', 'region_name']].drop_duplicates()
data_unique['region_name'] = data_unique['region_name'].apply(lambda x: bigger_regions.get(x, x))


data_unique_eu_non =  data[['wikidata_id', 'meta_region']].drop_duplicates()
data_unique_eu_non['meta_region'] = data_unique_eu_non['meta_region'].apply(lambda x: bigger_regions.get(x, x))

data_unique_eu_non = data_unique_eu_non[data_unique_eu_non['meta_region']!='antiquity']
data_unique_eu_non = data_unique_eu_non.rename(columns={'meta_region':'region_name'})

data_unique_all = pd.concat([data_unique, data_unique_eu_non])
data_unique_all = data_unique_all.groupby('region_name')['wikidata_id'].count().reset_index()


In [9]:


#total_row = data_unique_all[data_unique_all['region_name']=='europe']['wikidata_id']

total_numbers = data_unique_all[data_unique_all['region_name']=='europe']['wikidata_id'].values + data_unique_all[data_unique_all['region_name']=='non_europe']['wikidata_id'].values
total_numbers = total_numbers[0]

# Adding a new row for the total
total_row = {'region_name': 'Total number of scholars', 'wikidata_id': total_numbers}
data_unique_all = pd.concat([data_unique_all, pd.DataFrame([total_row])], ignore_index=True)

total_row = pd.Series(data=data_unique_all['wikidata_id'].values, index=data_unique_all['region_name'])


In [10]:



total_row.name = 'Total unique scholars'
pivot_table = pd.concat([pivot_table, pd.DataFrame([total_row], index=[total_row.name])])

pivot_table = pivot_table.astype(int)
pivot_table = pivot_table.sort_values('Total number of scholars', ascending=False)

top = pivot_table.head(1)
pivot_table = pd.concat([pivot_table,top])
pivot_table = pivot_table[1:]

pivot_table

,Ancient Mediterranean,Asia,British Islands,Eastern Europe,France,German world,Iberian Peninsula,Italy,Low countries,MENA,Nordic countries,europe,non_europe,Total number of scholars
theologian,71,16,301,449,555,2333,146,396,334,172,162,4676,259,4935
historian,209,208,206,264,486,612,284,532,201,275,106,2692,692,3384
philosopher,363,330,141,170,259,322,114,380,93,170,35,1514,863,2377
mathematician,80,88,135,114,192,316,56,225,89,109,44,1171,277,1448
astronomer,52,88,69,93,86,168,32,122,50,106,33,653,246,899
geographer,20,41,67,55,112,130,48,87,136,42,33,669,103,772
linguist,10,45,75,52,81,50,45,64,38,102,20,425,157,582
botanist,6,17,63,34,96,93,27,86,42,19,25,466,42,508
musicologist,24,4,25,14,45,74,26,73,15,8,5,277,36,313
physicist,13,3,34,26,62,80,3,41,26,12,11,283,28,311


In [11]:

df_poly = data[data['meta_occupation'].str.contains(' | ')]
total_unique_scholars = len(set(df_poly.wikidata_id))
df_poly['region_name'] = df_poly['region_name'].apply(lambda x: bigger_regions.get(x, x))
df_poly['meta_occupation'] = df_poly['meta_occupation'].apply(lambda x : x.split(' | '))
df_poly = df_poly.explode('meta_occupation')
df_poly = df_poly.drop_duplicates()
df_poly = df_poly[df_poly['meta_region']!='antiquity']

df_poly[(df_poly['region_name']=='Ancient Mediterranean')&(df_poly['meta_occupation']=='anthropologist')]

,wikidata_id,individual_name,region_code,region_name,birthyear,meta_occupation,meta_region
525,Q317942,Euhemerus,re_greek_world,Ancient Mediterranean,-400.0,anthropologist,non_europe
638,Q312548,Megasthenes,re_greek_world,Ancient Mediterranean,-350.0,anthropologist,non_europe


In [12]:
df_poly_all = df_poly.groupby(['meta_occupation','region_name'])['wikidata_id'].count().reset_index()

df_poly_eu = df_poly.drop('region_name', axis=1).drop_duplicates()
df_poly_eu = df_poly_eu[df_poly_eu['meta_region']!='antiquity']
df_poly_eu = df_poly_eu.rename(columns={'meta_region':'region_name'})
df_poly_eu= df_poly_eu.groupby(['meta_occupation','region_name'])['wikidata_id'].count().reset_index()


df_poly_final = pd.concat([df_poly_all, df_poly_eu])
df_poly_final = df_poly_final.rename(columns = {'wikidata_id':'poly'})
df_poly_final = df_poly_final.pivot(index='meta_occupation', columns='region_name', values = 'poly')
df_poly_final = df_poly_final.fillna(0)
df_poly_final['Total number of scholars'] = df_poly_final['europe'] + df_poly_final['non_europe']

df_poly_final

region_name,Ancient Mediterranean,Asia,British Islands,Eastern Europe,France,German world,Iberian Peninsula,Italy,Low countries,MENA,Nordic countries,europe,non_europe,Total number of scholars
meta_occupation,,,,,,,,,,,,,,
anatomist,2.0,1.0,13.0,3.0,14.0,21.0,1.0,16.0,13.0,1.0,7.0,88.0,4.0,92.0
anthropologist,2.0,3.0,4.0,3.0,10.0,6.0,6.0,6.0,1.0,2.0,0.0,36.0,7.0,43.0
archeologist,0.0,2.0,12.0,0.0,12.0,11.0,5.0,20.0,4.0,2.0,10.0,74.0,4.0,78.0
art theorist,0.0,1.0,0.0,1.0,3.0,3.0,0.0,6.0,4.0,0.0,0.0,17.0,1.0,18.0
astronomer,41.0,58.0,43.0,63.0,62.0,117.0,20.0,91.0,38.0,84.0,21.0,455.0,183.0,638.0
botanist,3.0,7.0,16.0,10.0,27.0,37.0,4.0,29.0,17.0,12.0,10.0,150.0,22.0,172.0
chemist,0.0,0.0,9.0,2.0,13.0,16.0,0.0,4.0,9.0,9.0,4.0,57.0,9.0,66.0
economist,0.0,3.0,9.0,2.0,7.0,5.0,6.0,7.0,2.0,1.0,1.0,39.0,4.0,43.0
epidemiologist,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,3.0,1.0,4.0


In [13]:
df_poly_unique = data[data['meta_occupation'].str.contains(' | ')]
df_poly_unique['region_name'] = df_poly_unique['region_name'].apply(lambda x: bigger_regions.get(x, x))

df_poly_unique_all = df_poly_unique.groupby(['region_name'])['wikidata_id'].nunique().reset_index()
df_poly_unique_eu = df_poly_unique.groupby(['meta_region'])['wikidata_id'].nunique().reset_index()
df_poly_unique_eu = df_poly_unique_eu.rename(columns={'meta_region':'region_name'})
df_poly_unique_eu = df_poly_unique_eu[df_poly_unique_eu['region_name']!='antiquity']

df_poly_unique_final = pd.concat([df_poly_unique_eu, df_poly_unique_all])

total_unique = len(set(df_poly_unique['wikidata_id']))

# Adding a new row for the total
total_row_unique = {'region_name': 'Total number of scholars', 'wikidata_id': total_unique}
df_poly_unique_final = pd.concat([df_poly_unique_final, pd.DataFrame([total_row_unique])], ignore_index=True)
df_poly_unique_final = df_poly_unique_final.set_index('region_name')

list_ordered = list(df_poly_final.columns)

df_poly_unique_final = df_poly_unique_final.reindex(list_ordered)
df_poly_unique_final = df_poly_unique_final['wikidata_id']
df_poly_unique_final.name = 'Total unique scholars'


df_poly_final = df_poly_final.append(df_poly_unique_final)
# df_poly_final = pd.concat([df_poly_final, df_poly_unique_final], axis=1)
# df_poly_final

/var/folders/lx/9b13r6ln63jb54_3xlf9b_380000gn/T/ipykernel_43708/3880735074.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_poly_final = df_poly_final.append(df_poly_unique_final)


In [14]:
df_poly_final

region_name,Ancient Mediterranean,Asia,British Islands,Eastern Europe,France,German world,Iberian Peninsula,Italy,Low countries,MENA,Nordic countries,europe,non_europe,Total number of scholars
meta_occupation,,,,,,,,,,,,,,
anatomist,2.0,1.0,13.0,3.0,14.0,21.0,1.0,16.0,13.0,1.0,7.0,88.0,4.0,92.0
anthropologist,2.0,3.0,4.0,3.0,10.0,6.0,6.0,6.0,1.0,2.0,0.0,36.0,7.0,43.0
archeologist,0.0,2.0,12.0,0.0,12.0,11.0,5.0,20.0,4.0,2.0,10.0,74.0,4.0,78.0
art theorist,0.0,1.0,0.0,1.0,3.0,3.0,0.0,6.0,4.0,0.0,0.0,17.0,1.0,18.0
astronomer,41.0,58.0,43.0,63.0,62.0,117.0,20.0,91.0,38.0,84.0,21.0,455.0,183.0,638.0
botanist,3.0,7.0,16.0,10.0,27.0,37.0,4.0,29.0,17.0,12.0,10.0,150.0,22.0,172.0
chemist,0.0,0.0,9.0,2.0,13.0,16.0,0.0,4.0,9.0,9.0,4.0,57.0,9.0,66.0
economist,0.0,3.0,9.0,2.0,7.0,5.0,6.0,7.0,2.0,1.0,1.0,39.0,4.0,43.0
epidemiologist,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,3.0,1.0,4.0


In [15]:
df_poly_unique_final

region_name
Ancient Mediterranean        146
Asia                         105
British Islands              198
Eastern Europe               192
France                       324
German world                 505
Iberian Peninsula             98
Italy                        328
Low countries                164
MENA                         178
Nordic countries              79
europe                      1888
non_europe                   429
Total number of scholars    2317
Name: Total unique scholars, dtype: int64

In [16]:
# # Adding a new row for the total
# total_row = {'meta_occupation': 'Total Unique Scholars', 'wikidata_id': total_unique_scholars}
# df_poly = pd.concat([df_poly, pd.DataFrame([total_row])], ignore_index=True)

# # Set 'meta_occupation' as index in df_poly
# df_poly.set_index('meta_occupation', inplace=True)

# list_ordered = list(pivot_table.index)
# list_ordered

# # Reindex df_poly according to list_ordered
# df_poly = df_poly.reindex(list_ordered)
# df_poly

In [17]:
# Convert df_poly_final values to string and prepend with parentheses
df_poly_formatted = df_poly_final.applymap(lambda x: f" ({int(x)})" if pd.notna(x) else "")

# Combine the original pivot_df with the new df_poly_formatted
result_df = pivot_table.astype(str) + df_poly_formatted


In [18]:


# Remove parentheses and extract the first numeric part for sorting
result_df['total_numeric'] = result_df['Total number of scholars'].str.extract('(\d+)').astype(int)

# Sort the dataframe by the 'total_numeric' column
result_df = result_df.sort_values(by='total_numeric', ascending=False)

# Drop the temporary numeric column if you don't need it after sorting
result_df = result_df.drop(columns=['total_numeric'])


# Display the result

top = result_df.head(1)
result_df = pd.concat([result_df,top])
result_df = result_df[1:]
result_df.index = result_df.index.str.capitalize()
result_df.to_csv('table_1_paper.csv')

import dataframe_image as dfi
# result_df = result_df.style.background_gradient()  # Adds a visual gradient for aesthetics
dfi.export(result_df, 'output_table_2.png')

In [28]:
result_df

,Ancient Mediterranean,Asia,British Islands,Eastern Europe,France,German world,Iberian Peninsula,Italy,Low countries,MENA,Nordic countries,europe,non_europe,Total number of scholars
Theologian,71 (25),16 (4),301 (68),449 (81),555 (137),2333 (261),146 (33),396 (98),334 (61),172 (36),162 (29),4676 (768),259 (65),4935 (833)
Historian,209 (48),208 (33),206 (43),264 (60),486 (95),612 (146),284 (42),532 (85),201 (50),275 (60),106 (27),2692 (548),692 (141),3384 (689)
Philosopher,363 (110),330 (39),141 (84),170 (69),259 (132),322 (160),114 (33),380 (150),93 (53),170 (71),35 (18),1514 (699),863 (220),2377 (919)
Mathematician,80 (68),88 (56),135 (72),114 (79),192 (84),316 (177),56 (25),225 (124),89 (51),109 (77),44 (24),1171 (636),277 (201),1448 (837)
Astronomer,52 (41),88 (58),69 (43),93 (63),86 (62),168 (117),32 (20),122 (91),50 (38),106 (84),33 (21),653 (455),246 (183),899 (638)
Geographer,20 (15),41 (15),67 (24),55 (25),112 (41),130 (47),48 (20),87 (32),136 (34),42 (21),33 (2),669 (225),103 (51),772 (276)
Linguist,10 (0),45 (18),75 (19),52 (17),81 (23),50 (31),45 (13),64 (22),38 (14),102 (26),20 (6),425 (145),157 (44),582 (189)
Botanist,6 (3),17 (7),63 (16),34 (10),96 (27),93 (37),27 (4),86 (29),42 (17),19 (12),25 (10),466 (150),42 (22),508 (172)
Musicologist,24 (19),4 (2),25 (6),14 (3),45 (12),74 (18),26 (4),73 (10),15 (5),8 (6),5 (2),277 (60),36 (27),313 (87)
Physicist,13 (12),3 (2),34 (29),26 (15),62 (46),80 (50),3 (2),41 (32),26 (22),12 (11),11 (10),283 (206),28 (25),311 (231)


In [24]:


result_df_fil = result_df[['europe', 'non_europe', 'Total number of scholars']]
#df_styled = result_df_fil.style.background_gradient()  # Adds a visual gradient for aesthetics
dfi.export(result_df_fil, 'output_table_1.png')

In [27]:

result_df_fil

,europe,non_europe,Total number of scholars
Theologian,4676 (768),259 (65),4935 (833)
Historian,2692 (548),692 (141),3384 (689)
Philosopher,1514 (699),863 (220),2377 (919)
Mathematician,1171 (636),277 (201),1448 (837)
Astronomer,653 (455),246 (183),899 (638)
Geographer,669 (225),103 (51),772 (276)
Linguist,425 (145),157 (44),582 (189)
Botanist,466 (150),42 (22),508 (172)
Musicologist,277 (60),36 (27),313 (87)
Physicist,283 (206),28 (25),311 (231)
